In [31]:
## Make a neuron spike just once and then maintain it on the refractary period.

## Create a LIF _Process_

In [32]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort

class NEURON(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get("shape", (1,))
        bias = kwargs.pop("bias", 0)
        vth = kwargs.pop("vth", 10)

        self.shape = shape
        self.a_in = InPort(shape=shape)
        self.s_out = OutPort(shape=shape)
        self.w = Var(shape=shape, init=0)
        self.u = Var(shape=shape, init=0)
        self.v = Var(shape=shape, init=0)
        self.aux = Var(shape=shape, init=0)
        self.aux2 = Var(shape=shape, init=0)
        self.bias = Var(shape=shape, init=bias)
        self.vth = Var(shape=(1,), init=vth)

class NEURON2(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get("shape", (1,))
        bias = kwargs.pop("bias", 0)
        vth = kwargs.pop("vth", 10)

        self.shape = shape
        self.a_in = InPort(shape=shape)
        self.s_out = OutPort(shape=shape)
        self.w = Var(shape=shape, init=0)
        self.u = Var(shape=shape, init=0)
        self.v = Var(shape=shape, init=0)
        self.bias = Var(shape=shape, init=bias)
        self.vth = Var(shape=(1,), init=vth)

## Create a Python _LeafProcessModel_ that implements the LIF _Process_

Now, we will create a Python _ProcessModel_, or _PyProcessModel_, that runs on a CPU compute resource and implements the LIF _Process_ behavior.

#### Setup

We begin by importing the required Lava classes.
First, we setup our compute resources (CPU) and our _SyncProtocol_. A _SyncProtocol_ defines how and when parallel _Processes_ synchronize. Here we use the _LoihiProtoicol_ which defines the synchronization phases required for execution on the Loihi chip, but users may also specify a completely asynchronous protocol or define a custom _SyncProtocol_. The decorators imported will be necessary to specify the resource _Requirements_ and _SyncProtocol_ of our _ProcessModel_. 

In [33]:
import numpy as np
from lava.magma.core.decorator import implements, requires
from lava.magma.core.resources import CPU
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol

Now we import the parent class from which our _ProcessModel_ inherits, as well as our required _Port_ and _Variable_ types. _PyLoihiProcessModel_ is the abstract class for a Python _ProcessModel_ that implements the _LoihiProtocol_. Our _ProcessModel_ needs _Ports_ and _Variables_ that mirror those the LIF _Process_. The in-ports and out-ports of a Python _ProcessModel_ have types _PyInPort_ and _PyOutPort_, respectively, while variables have type _LavaPyType_.

In [34]:
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
from lava.proc.lif.process import LIF

#### Defining a _PyLifModel_ for LIF

In [35]:
import numpy as np
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.proc.lif.process import LIF


@implements(proc=NEURON, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyLifModel1(PyLoihiProcessModel):
    a_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float)
    s_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    w: np.ndarray = LavaPyType(np.ndarray, float)
    u: np.ndarray = LavaPyType(np.ndarray, float)
    v: np.ndarray = LavaPyType(np.ndarray, float)
    aux: np.ndarray = LavaPyType(np.ndarray, float)
    aux2: np.ndarray = LavaPyType(np.ndarray, float)
    bias: np.ndarray = LavaPyType(np.ndarray, float)
    vth: float = LavaPyType(float, float)

    def run_spk(self):
        a_in_data=np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,2,4,5])
        self.u=a_in_data

        for index in range(len(self.v)):
            if (self.aux[index]<1):
                if(self.time_step==1):
                    self.v[index] = self.v[index] + self.u[index]*self.time_step+self.bias[index]
                else:
                    self.v[index] = self.v[index] + self.bias[index]
        s_out = self.v >= self.vth
        self.aux[:]+=s_out
        self.aux2[s_out]=self.time_step
        self.v[s_out] = 0  # Reset voltage to 0
        self.s_out.send(s_out)




@implements(proc=NEURON2, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyLifModel2(PyLoihiProcessModel):
    a_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float)
    s_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    w: np.ndarray = LavaPyType(np.ndarray, float)
    u: np.ndarray = LavaPyType(np.ndarray, float)
    v: np.ndarray = LavaPyType(np.ndarray, float)
    bias: np.ndarray = LavaPyType(np.ndarray, float)
    vth: float = LavaPyType(float, float)

    def run_spk(self):
        a_in_data2=self.a_in.recv()

        self.w[:] += a_in_data2[:]


        s_out = self.v >= self.vth
        self.v[s_out] = 0  # Reset voltage to 0
        self.s_out.send(s_out)

#### Compile and run _PyLifModel_

In [36]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

num_steps_per_input = 1000

lif = NEURON(shape=(10,), bias=0.2,num_steps=num_steps_per_input, vth=6)
lif2 = NEURON2(shape=(10,))
run_cfg = Loihi1SimCfg()

lif.s_out.connect(lif2.a_in)


lif.run(condition=RunSteps(num_steps=num_steps_per_input), run_cfg=run_cfg)
print("in 1 : ",lif.u.get())
print("out 1: ",lif.v.get())
print("time: ",lif.aux2.get())
print("in 2 : ",lif2.w.get())


in 1 :  [0.  0.1 0.2 0.3 0.4 0.5 0.6 2.  4.  5. ]
out 1:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
time:  [30. 30. 29. 29. 28. 28. 27. 20. 10.  5.]
in 2 :  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
